In [1]:
import os
import re
import logging

In [2]:
%run uploads_images_to_ghost.ipynb

In [3]:
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    filename='log_preprocessing_images.log'
)

In [4]:
origin_directory = '/Users/42piratas/Playground/gatsby-to-ghost/origin/'
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']

In [5]:
def rename_img(old_img_name, sub_dir, sub_dir_path):
    
    new_img_name = sub_dir + "-" + file.lower()
    old_img_path = os.path.join(sub_dir_path, old_img_name)
    new_img_path = os.path.join(sub_dir_path, new_img_name)
    os.rename(old_img_path, new_img_path)    
    
    logging.info(f'img renamed: {sub_dir}: {old_img_name} ---> {new_img_name}')
    
    return old_img_path, old_img_name, new_img_path, new_img_name

In [6]:
# Define the function to replace image paths with URLs
def replace_image_paths_with_url(sub_dir, sub_dir_path, old_img_name, image_url):
    md_file_path = os.path.join(sub_dir_path, 'index.md')

    # Read the content of the Markdown file
    with open(md_file_path, 'r') as file:
        md_content = file.read()

    # Define a regular expression pattern to match image locations
    image_path_pattern = re.compile(r"!\[[^\]]*\]\((.*?)\)")

    # Use re.sub to replace the old image locations with the new image URL
    def replace_image(match):
        image_path = match.group(1)
        if image_path.endswith(old_img_name) or image_path.endswith(f"./{old_img_name}"):
            new_path = image_url
            return match.group().replace(image_path, new_path)
        else:
            return match.group()

    modified_content = image_path_pattern.sub(replace_image, md_content)

    # Special handling for the hero image
    modified_content = modified_content.replace(f"hero: ./{old_img_name}", f"hero: {image_url}")

    # Write the modified content back to the file
    with open(md_file_path, 'w') as file:
        file.write(modified_content)
        logging.info(f'md updated: {sub_dir}: {old_img_name} ---> {image_url}')

In [7]:
for sub_dir in os.listdir(origin_directory):
    
    sub_dir_path = os.path.join(origin_directory, sub_dir)
    
    if os.path.isdir(sub_dir_path):
        
        for file in os.listdir(sub_dir_path):
            file_extension = os.path.splitext(file)[-1].lower()    
            
            if file_extension in image_extensions:
                old_img_path, old_img_name, new_img_path, new_img_name = rename_img(file, sub_dir, sub_dir_path)
                image_url = uploads_img(sub_dir, new_img_path, new_img_name)

                if not image_url:
                    logging.error(f'ERROR IMAGE ---> {new_img_path}')
                    continue

                replace_image_paths_with_url(sub_dir, sub_dir_path, old_img_name, image_url)